<h2><left>Hello, This Notebook Contains example for GSP Algorithm </left></h2>
<h4><left>GSP algorithm used in Data-Mining field to extrat frequent patterns from Transactions.</left></h4>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul><li><h4> Contents: </h4></li>
    <ul>
        <li><a href="#ref1">Data Preparation</a></li>
        <li><a href="#ref2">Auxeliary Methods</a></li>
        <li><a href="#ref4">GSP</a></li>
        <li><a href="#ref5">Results</a></li>
    </ul></ul>
</div>
<hr>

In [128]:
import pandas as pd
import re
import itertools
import numpy as np

<a id="ref1"></a>
<h3><left> - Data Preparation:</left></h3>
<h4><left> - Reading and Visualizing.</left></h4>
  

In [2]:
DATA_PATH = '../Data/paths_finished.tsv'

In [3]:
data = pd.read_csv(DATA_PATH,sep='\t',names = ['hashedIpAddress',   'timestamp ',  'durationInSec',   'path',   'rating'])
data

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0
...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,Yagan;Ancient_Egypt;Civilization,NaN
51314,2ef7ac844cefda58,1300254138,165,Yagan;Folklore;Brothers_Grimm;<;19th_century;C...,3.0
51315,12863abb7887f890,1385095372,228,Yagan;Australia;England;France;United_States;T...,NaN
51316,19f8284371753362,1298792567,56,Yarralumla%2C_Australian_Capital_Territory;Aus...,1.0


In [4]:
data.sample(20)

,hashedIpAddress,timestamp,durationInSec,path,rating
49408,0d57c8c57d75e2f5,1291378963,47,Topology;Geometry;Physics;Light,1.0
47940,5464b0ce26a4dc27,1248899187,71,Tehran;Greek_mythology;Mythology;The_Lord_of_t...,1.0
4189,6d3a885a4dd06d26,1350922886,75,Elizabeth_II_of_the_United_Kingdom;United_King...,4.0
22090,2dba694112f702af,1225139095,126,Nuclear_power;Global_warming;Ocean;Dolphin,NaN
17817,5bb5836b425e6bfe,1324130348,195,Wine;Great_Britain;French_language;English_lan...,NaN
11327,79e9b0c465d376aa,1386180216,13,Amazon_River;Nile,NaN
30283,27bb85bf43c94394,1237509177,49,Florida_East_Coast_Railway;Coal;Electricity;Al...,NaN
27718,13fa1c2039bd5f0c,1311916512,205,Wind;Radar;World_War_II;Nuclear_weapon;Nuclear...,1.0
41704,272f848a2c58b002,1248844924,34,Koryaksky;Russia;United_States;September_11%2C...,1.0
7821,3971438a14ed8c94,1250478536,246,The_Godfather;France;United_Kingdom;Airbus,3.0


<a id="ref2"></a>
<h3><left>- Aux Methods.</left></h3>
<h4><left>- we will using this methods in all project stages(data-cleaning, algorithm).</left></h4>

In [51]:
from collections import Counter 
import numpy as np

# methods for cleaning data:
def do_some_preprocess(path,pattern):
    items = str(path).lower().split(';')
    items_ = [re.sub(pattern,'',item) for item in items]
    return ';'.join([item for item in items_ if item !=''])


def drop_values(path,drop_list):
    items = str(path).lower().split(';')
    items_ = ';'.join([item for item in items if item not in drop_list and item != ''])
    return items_

def merge_as_string(bigram):
     return ';'.join(item for item in bigram)
    
def pathes_to_string(pathes):
    return '&'.join([path for path in pathes])

def get_words(pathes):
    str_pathes = ''.join([path for path in pathes])
    return pd.Series(str_pathes.split(';'))

# methods for patterns processing:
def count_patterns(pattern,pathes,counter):
    for i in range(len(pathes)):
        if pattern in pathes[i]:#re.match(pattern,pathes[i]):
            counter[pattern]+=1

def get_n_gram(grams,n=3):
    first_term = []
    second_term = []
    n_grams = []
    n_grams_str = []
    if n <= 2 or len(grams[0]) < n-1:
        raise ValueError()
    i = n-2
   
    for gram in grams:
        first_term.append(gram[-i:])
        second_term.append(gram[:i])
        
    for ind_1,item_1 in enumerate(first_term):
        for ind_2, item_2 in enumerate(second_term):
            if item_1 == item_2:
                t = grams[ind_1] + grams[ind_2][i:]
                n_grams.append(t)
                n_grams_str.append(';'.join(e for e in t))
    n_grams= pd.Series(n_grams)
    n_grams_str= pd.Series(n_grams_str)
    
    return n_grams,n_grams_str



def get_bigrams(unigrams):
    bigrams = list(itertools.product(*[unigrams]*2))
    bigrams = pd.Series(bigrams)
    bigrams_str = bigrams.apply(merge_as_string)
    return bigrams,bigrams_str

def drop_less_supports(counter,min_support):
    counter = pd.Series(counter)
    result = counter[counter > min_support]
    result = [ind.split(';') for ind in result.index]
    return result

<a id="ref4"></a>
<h3><left>- GSP Algorithm.</left></h3>
<h4><left>- loop over levels to extract patterns.</left></h4>

In [126]:
def gsp(data,sampels_num,max_levels,min_support=10):
    #data must have 'path' label
    all_patterns = []
    pat = r'[^A-Za-z_1-9]'
    try:
        pathes = data.path.apply(do_some_preprocess,pattern= pat)[:sampels_num]
        pathes_str = pathes_to_string(pathes)
    except AttributeError:
        print('data isn\'t acceptable! ')
       
    words = get_words(pathes)
    counts = words.value_counts()
    unigrams = counts[counts > min_support]

    drop_list = counts[counts < min_support]
    pathes = pathes.apply(drop_values,drop_list=drop_list)
    
    patterns, patterns_s = get_bigrams(unigrams.index)#[:sampels_num]
    level = 2
    while level < max_levels:
        print("Level: ",level)
        co = Counter()
        patterns_s.apply(count_patterns,pathes = pathes,counter = co)#[:sampels_num]
        fn = drop_less_supports(co,min_support=min_support)
        if len(fn) is 0:
            break
        level +=1
        all_patterns.append(fn)
        patterns, patterns_s = get_n_gram(fn,n=level)
        
    return all_patterns

<a id="ref5"></a>
<h3><left> - Results</left></h3>

In [127]:
min_support = 2
sampels_num = 100
max_levels = 10

gsp(data,min_support=min_support,max_levels=max_levels,sampels_num=sampels_num)

Level:  2
Level:  3
Level:  4


E:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:50: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
E:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
E:\Applications\Anaconda\lib\site-packages\ipykernel_launcher.py:66: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


[[['time', 'light'],
  ['time', 'isaac_newton'],
  ['physics', 'science'],
  ['physics', 'speed_of_light'],
  ['light', 'rain'],
  ['color', 'rain'],
  ['science', 'chemistry'],
  ['isaac_newton', 'rain'],
  ['internet', 'world_wide_web'],
  ['ultraviolet', 'color'],
  ['earth27s_atmosphere', 'ultraviolet'],
  ['speed_of_light', 'rain'],
  ['computer', 'internet'],
  ['africa', 'atlantic_slave_trade'],
  ['el_nic3b1osouthern_oscillation', 'global_warming'],
  ['15th_century', '16th_century'],
  ['us_state', 'united_states'],
  ['fish', 'fishing'],
  ['fish', 'fish'],
  ['potassium', 'potassium'],
  ['weather', 'sunlight'],
  ['weather', 'sun'],
  ['atlantic_slave_trade', 'africa']],
 [['time', 'light', 'rain'],
  ['time', 'isaac_newton', 'rain'],
  ['physics', 'speed_of_light', 'rain'],
  ['ultraviolet', 'color', 'rain'],
  ['africa', 'atlantic_slave_trade', 'africa']]]

<h3><left>Thank You...</left></h3>